[![Jupyter Notebook](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamindb/blob/main/docs/registries.ipynb)

# Query & search registries

This guide walks through different ways of querying & searching LaminDB registries.

In [ ]:
# pip install 'lamindb[bionty]'
!lamin init --storage ./test-registries --modules bionty

Let's start by creating a few exemplary datasets and saving them into a LaminDB instance using, e.g., {func}`~lamindb.examples.ingest_mini_immuno_datasets`.

In [ ]:
import lamindb as ln

ln.track("Wc8F4siRSKMZ")

ln.Artifact(ln.core.datasets.file_jpg_paradisi05(), key="images/my_image.jpg").save()
ln.Artifact(ln.core.datasets.file_fastq(), key="raw/my_fastq.fastq.gz").save()
ln.Artifact.from_df(ln.core.datasets.df_iris(), key="iris.parquet").save()
ln.examples.ingest_mini_immuno_datasets()

## Get an overview

The easiest way to get an overview over all artifacts is by typing {meth}`~lamindb.Artifact.df`, which returns the 100 latest artifacts in the {class}`~lamindb.Artifact` registry.

In [ ]:
ln.Artifact.df()

You can include fields from other registries.

In [ ]:
ln.Artifact.df(
    include=[
        "created_by__name",
        "ulabels__name",
        "cell_types__name",
        "feature_sets__itype",
        "suffix",
    ]
)

You can include information about which artifact measures which `feature`.

In [ ]:
df = ln.Artifact.df(features=True)
ln.view(df)  # optionally use ln.view() to see dtypes

The flattened table that includes information from all relevant registries is easier to understand than the normalized data.

In [ ]:
ln.view()

## Auto-complete records

For registries with less than 100k records, auto-completing a `Lookup` object is the most convenient way of finding a record.

In [ ]:
import bionty as bt

# query the database for all ulabels or all cell types
ulabels = ln.ULabel.lookup()
cell_types = bt.CellType.lookup()

:::{dropdown} Show me a screenshot

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/lgRNHNtMxjU0y8nIagt7.png" width="400px">

:::

With auto-complete, we find a ulabel:

In [ ]:
study1 = ulabels.experiment_1
study1

## Get one record

{class}`~lamindb.models.SQLRecord.get` errors if more than one matching records are found.

In [ ]:
print(study1.uid)

# by uid
ln.ULabel.get(study1.uid)

# by field
ln.ULabel.get(name="Experiment 1")

## Query records by fields

Filter for all artifacts annotated by a ulabel:

In [ ]:
ln.Artifact.filter(ulabels=study1).df()

To access the results encoded in a filter statement, execute its return value with one of:

- {meth}`~lamindb.models.QuerySet.df`: A pandas `DataFrame` with each record in a row.
- {meth}`~lamindb.models.QuerySet.all`: A {class}`~lamindb.models.QuerySet`.
- {meth}`~lamindb.models.QuerySet.one`: Exactly one record. Will raise an error if there is none. Is equivalent to the `.get()` method shown above.
- {meth}`~lamindb.models.QuerySet.one_or_none`: Either one record or `None` if there is no query result.

```{note}

{meth}`~lamindb.models.SQLRecord.filter` returns a {class}`~lamindb.models.QuerySet`.

The registries in LaminDB are Django Models and any [Django query](https://docs.djangoproject.com/en/stable/topics/db/queries/) works.

LaminDB re-interprets Django's API for data scientists.

```

```{dropdown} What does this have to do with SQL?

Under the hood, any `.filter()` call translates into a SQL select statement.

LaminDB's registries are object relational mappers (ORMs) that rely on Django for all the heavy lifting.

Of note, `.one()` and `.one_or_none()` are the two parts of LaminDB's API that are borrowed from SQLAlchemy. In its first year, LaminDB built on SQLAlchemy.

```

## Query datasets by features

The `Artifact` registry is the only registry that additionally allows to query by features.

In [ ]:
ln.Artifact.filter(perturbation="DMSO").df(features=True)

You can also query for nested dictionary-like features.

In [ ]:
ln.Artifact.filter(study_metadata__detail1="123").df()

In [ ]:
ln.Artifact.filter(study_metadata__detail2=2).df()

You can query for whether a dataset is annotated or not annotated by a feature.

In [ ]:
ln.Artifact.filter(perturbation__isnull=True).df()

In [ ]:
ln.Artifact.filter(perturbation__isnull=False).df()

## Query runs by parameters

Here is an example for querying by parameters: {ref}`query-by-run-parameters`.

## Search for records

You can search every registry via {meth}`~lamindb.models.SQLRecord.search`. For example, the `Artifact` registry.

In [ ]:
ln.Artifact.search("iris").df()

Here is more background on search and examples for searching the entire cell type ontology: {doc}`/faq/search` 

## Query related registries

Django has a double-under-score syntax to filter based on related tables.

This syntax enables you to traverse several layers of relations and leverage different comparators.

In [ ]:
ln.Artifact.filter(created_by__handle__startswith="testuse").df()

The filter selects all artifacts based on the users who ran the generating notebook. Under the hood, in the SQL database, it's joining the artifact table with the user table.

Another typical example is querying all datasets that measure a particular feature. For instance, which datasets measure `"CD8A"`. Here is how to do it:

In [ ]:
cd8a = bt.Gene.get(symbol="CD8A")
# query for all feature sets that contain CD8A
feature_sets_with_cd8a = ln.Schema.filter(genes=cd8a).all()
# get all artifacts
ln.Artifact.filter(feature_sets__in=feature_sets_with_cd8a).df()

Instead of splitting this across three queries, the double-underscore syntax allows you to define a path for one query.

In [ ]:
ln.Artifact.filter(feature_sets__genes__symbol="CD8A").df()

## Filter operators

You can qualify the type of comparison in a query by using a comparator.

Below follows a list of the most import, but Django supports about [two dozen field comparators](https://docs.djangoproject.com/en/stable/ref/models/querysets/#field-lookups) `field__comparator=value`.

### and

In [ ]:
ln.Artifact.filter(suffix=".h5ad", ulabels=study1).df()

### less than/ greater than

Or subset to artifacts greater than 10kB. Here, we can't use keyword arguments, but need an explicit where statement.

In [ ]:
ln.Artifact.filter(ulabels=study1, size__gt=1e4).df()

### in

In [ ]:
ln.Artifact.filter(suffix__in=[".jpg", ".fastq.gz"]).df()

### order by

In [ ]:
ln.Artifact.filter().order_by("created_at").df()

In [ ]:
# reverse ordering
ln.Artifact.filter().order_by("-created_at").df()

In [ ]:
ln.Artifact.filter().order_by("key").df()

In [ ]:
# reverse ordering
ln.Artifact.filter().order_by("-key").df()

### contains

In [ ]:
ln.Transform.filter(description__contains="search").df().head(5)

And case-insensitive:

In [ ]:
ln.Transform.filter(description__icontains="Search").df().head(5)

### startswith

In [ ]:
ln.Transform.filter(description__startswith="Query").df()

### or

In [ ]:
ln.Artifact.filter(ln.Q(suffix=".jpg") | ln.Q(suffix=".fastq.gz")).df()

### negate/ unequal

In [ ]:
ln.Artifact.filter(~ln.Q(suffix=".jpg")).df()